# References

In [ ]:
# For this implementation of the Decision Tree we made use of the following source code, and improved it to make it faster:
# The code is taken from a Youtube tutorial, given by Josh Gordon. The code he uses in the tutorial can be found on
# https://github.com/random-forests/tutorials/blob/master/decision_tree.py
# Last accessed on 22-06-2020. 

# We changed this code to be able to add the differential privacy
# The theory of adding privacy is obtained from the following papers:
# 1. S. Fletcher, MD Z. Islam, 'Decision Tree Classification with Differential Privacy: A Survey'
# The paper can be found on https://arxiv.org/pdf/1611.01919.pdf, Last accessed on 22-06-2020. 

# 2. G. Jgannathan, K. Pillaipakkamnatt, R. N. Wright, 'A Practical Differentially Private Random Decision Tree Classifier'


# Preprocessing

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from time import time

# This target is the name of the column that contains the label. In the SPECT_Heart dataset this is the 'Diagnosis' column.
target = 'Diagnosis'

def prepare_data(location_training_data, location_testing_data=None):
    # Define the headings to be used
    headings = ['Diagnosis','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13','F14','F15','F16','F17','F18','F19','F20','F21','F22']

    # Read in the official training and testing data separately
    df = pd.read_csv(location_training_data, names=headings)

    if location_testing_data is not None:
        data_test = pd.read_csv(location_testing_data, names=headings, skiprows=1)

        # Combine both data sets into one data set
        df = df.append(data_test)

    # Apply some pre-processing
    # For an attribute that contains only numerical values do:
    # df[attribute_name] = df[attribute_name].astype('int').apply(pd.to_numeric, downcast="unsigned")
    # For an attribute that contains only categorical values do:
    # df[attribute_name] = df[attribute_name].astype('category').str.strip()

    df['Diagnosis'] = df['Diagnosis'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F1'] = df['F1'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F2'] = df['F2'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F3'] = df['F3'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F4'] = df['F4'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F5'] = df['F5'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F6'] = df['F6'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F7'] = df['F7'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F8'] = df['F8'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F9'] = df['F9'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F10'] = df['F10'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F11'] = df['F11'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F12'] = df['F12'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F13'] = df['F13'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F14'] = df['F14'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F15'] = df['F15'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F16'] = df['F16'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F17'] = df['F17'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F18'] = df['F18'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F19'] = df['F19'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F20'] = df['F20'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F21'] = df['F21'].astype('int').apply(pd.to_numeric, downcast="unsigned")
    df['F22'] = df['F22'].astype('int').apply(pd.to_numeric, downcast="unsigned")

    # Convert every '?' to NaN, so Panda's built-in functions can be used
    df = df.where(df != '?')

    # Try to fill missing values with the mean (this only works for numerical attributes)
    df = df.fillna(df.mean())

    # Get the number of records in the data set
    n_records_before = df.shape[0]

    # If values are still missing (they must be categorical attributes), drop the rows with missing data
    df = df.dropna()

    # Get the number of records in the data set
    n_records_after = df.shape[0]

    # Print how many records containing NaN values got dropped
    n_records_dropped = n_records_before - n_records_after
    print(n_records_dropped, "records were dropped due to missing values.")
    print("This is", round(n_records_dropped / n_records_before * 100, 1), "% of the entire data set.")
    print("The resulting data set contains", n_records_after, "records.")

    # Reset the indices
    df = df.reset_index(drop=True)

    return df

def cross_validation_train_test_split(data_set, buckets_location, bucket_number):
    # Read all the bucket indices for the specified bucket number from the provided CSV file
    bucket_indices = pd.read_csv(buckets_location).iloc[bucket_number].dropna().values

    # Split the entire data set into testing and training data
    data_test = data_set.iloc[bucket_indices]
    data_train = data_set.drop(bucket_indices, axis='index')

    # Reset the indices of the data
    data_test.reset_index(drop=True, inplace=True)
    data_train.reset_index(drop=True, inplace=True)

    return data_train, data_test


# Load and prepare the training and testing data
# When there are both a training file and a test file pass both as parameters, otherwise pass the whole file as training_data.

# Depending on where the datasets are stored it might be that the path is not correct, but the suffix is correct for the SPECT_Heart dataset.

all_data = prepare_data(location_training_data='../datasets/SPECT_Heart/SPECT.train', location_testing_data='../datasets/SPECT_Heart/SPECT.test')

# Define the location where the buckets are stored
buckets_loc = '../datasets/SPECT_Heart/buckets_SPEC.csv'

# Classifier

In [ ]:
class Leaf:
    # A class to return the Leaf node that contains the label.
    def __init__(self, dataset, target):
        self.predictions = dataset[target].value_counts()


class Decision_Node:
    # A class to return the Decision node, that contains a question and 2 branches, based on this question. 
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        
# Function to determine if a value is numeric. 
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)


class Question:
    # Class that represents the question, it contains the attribute (column) and the value of that attribute (value)
    def __init__(self, column, value):
        self.column = column
        self.value = value

    # Function to check if the example row matches the question, to determine the branch. 
    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
class DecisionTree:
    
    # The Decision Tree classifier class. It takes as input a parameter epsilon, which is optional.
    # Epsilon must be greater or equal then 0, so for example 1*10^-10. 
    def __init__(self, e=None):
        self._e = e
        return
    
    def laplace(self, mean, scale):
        # Sample the Laplace distribution to generate some noise as explained in the paper
        
        return np.random.laplace(loc=mean, scale=scale)

    def gini(self, dataset):
        # Return Gini index, as explained in the paper
        
        # No privacy, so the normal variant
        if self._e == None:
            return 1 - dataset[self._target].value_counts().div(dataset.shape[0]).pow(2).sum()
        
        # In this case a value for epsilon is entered. 
        
        # Compute the number of attributes that there is left, as we drop the attribute when we
        # found a question, and the data is binary the number of attributes is equal to the number
        # of columns in the dataset - 1, because we don't count the label. 
        n_attributes = dataset.shape[1] - 1
        
        # Compute the scale of the laplacian noise based on the epsilon value, number of attributes
        # and the depth of the tree, as explained in the paper.
        scale = 1 / (self._e / (n_attributes * (n_attributes+1)))
        
        # Compute the counts of the labels and add the noise to these counts. 
        noisy_label_counts = dataset[self._target].value_counts().add(self.laplace(0, scale))

        # The noise can be negative, when this is the case we set the noise to be equal to 0.0001, such that is is almost 0.
        noisy_label_counts.loc[noisy_label_counts < 0] = 0.0001
        
        # Compute the sum of the noisy labels
        noisy_sum = noisy_label_counts.sum()

        # Prevent division by zero with the if statement
        if noisy_sum > 0:
            noisy_label_counts = noisy_label_counts.div(noisy_sum)
  
        # Compute the impurity of the dataset. 
        impurity = 1 - noisy_label_counts.pow(2).sum()
         
        return impurity

    # Compute the information gain as explained in the paper.
    def info_gain(self, left, right, current_uncertainty):
        p = (float(left.shape[0]) / float(left.shape[0] + right.shape[0]))
        return current_uncertainty - (p * self.gini(left)) - ((1 - p) * self.gini(right))

    # Function to partition the data into the true_branch and the false_branch.
    # The data is splitted based on the question.
    # equal or not equal to the question value when the attribute is categorical
    # greater/equal or smaller than the question value when the attribute is numerical. 
    def partition(self, dataset, question):

        if dataset[question.column].dtype == 'object':
            true_branch = dataset.loc[dataset[question.column] == question.value]
            false_branch = dataset.loc[dataset[question.column] != question.value]
        else: 
            true_branch = dataset.loc[dataset[question.column] >= question.value]
            false_branch = dataset.loc[dataset[question.column] < question.value]
        
        # Return the true and false branch but drop the column we used to split the data on.
        return true_branch.drop(question.column, axis='columns'), false_branch.drop(question.column, axis='columns')

    # Function to find the best question to split the data on. 
    def find_best_split(self, feature, dataset, current_uncertainty):
        
        # Take one of the attributes
        feature_values = feature.unique()

        # Create an empty series object, with the size of the number of attribute values for that attribute. 
        gain_s = pd.Series(0, index=feature_values)

        # For all the possible values of the attribute, 
        # create a question using that attribute value,
        # partitite the dataset based on this question,
        # compute the information gain for this split,
        # store the information gains,
        # return the question with the highest information gain,
        # and return also the information gain itself. 
        for value in feature_values:
            question = Question(feature.name, value)
            true_branch, false_branch = self.partition(dataset, question)

            if true_branch.shape[0] == 0 or false_branch.shape[0] == 0:
                continue

            gain_s.loc[value] = self.info_gain(true_branch, false_branch, current_uncertainty)

        return pd.Series([gain_s.max(), gain_s.idxmax()], index=['gini', 'question'])

    def build_tree(self, dataset, target):
        # This function computes the subtrees for 1 node, based on the best question.
        
        self._target = target
        
        # Compute the best question for all the possible attributes
        # So per attribute 1 best question left.
        feature_gains = dataset.drop(self._target, axis='columns').apply(self.find_best_split, axis='index', args=[dataset, self.gini(dataset)])

        # Take the best question of all the possible attributes
        # So 1 question left
        best_feature = feature_gains.loc['gini'].astype('float').idxmax()
        # Get the best gain, of splitting the dataset on this question
        gain = feature_gains[best_feature]['gini']
        # Get the value of the question itself
        question_value = feature_gains[best_feature]['question']
        
        # Create a question object
        question = Question(best_feature, question_value)
        
        # If we don't gain anything for the best possible split we can stop and return the label.
        if gain == 0:
            return Leaf(dataset, self._target)

        # If we do gain something for this partition, compute the true and false branch again. 
        true_branch, false_branch = self.partition(dataset, question)
        
        # Recursively compute the rest of the tree, by calling build_tree with the true and false branch. 
        tb = self.build_tree(true_branch, self._target)
        fb = self.build_tree(false_branch, self._target)

        # Return the tree in a particular format. 
        return Decision_Node(question, tb, fb)

    # A function to classify the data, when the training of the algorithm is completed.
    # This function takes an instance (row) of the dataset and a (sub)tree (node).
    # Then it compares the attribute in the row with the question, to determine if
    # the algorithm should take the true or false branch. 
    def classify(self, row, node):
        # Base case: we've reached a leaf
        if isinstance(node, Leaf):
            return node.predictions

        if node.question.match(row):
            return self.classify(row, node.true_branch)
        else:
            return self.classify(row, node.false_branch)

# Classify the test instances

In [ ]:
%%time

# This code might look very long and complicated, but is actually pretty straightforward 
# and a repitition of the same testcase, with a different value.

# We do 10-fold crossvalidation so we loop over our testcases 10 times, therefore the value r = 10

r = 10

# Create variables to store / compute the total accuracy over the 10 folds. 
accuracy_nn = 0
accuracy_1 = 0
accuracy_075 = 0
accuracy_050 = 0
accuracy_025 = 0
accuracy_010 = 0
accuracy_005 = 0
accuracy_001 = 0
accuracy_0005 = 0
accuracy_0001 = 0
accuracy_0 = 0

# instantiate the tree object
tree = None

# function to classify an instance of the dataset with the tree.
def predict(row):
        return classifier.classify(row, tree).index[0]

# Loop over the test-cases r times (r = 10, so 10-fold cross-validation)
for i in range(r):
    print()
    
    # Retrieve the training and testing data according to a specific bucket number (starts from 0)
    # Since we need a different bucket for every fold we set the bucket number equal to the loop counter.
    training_data, testing_data = cross_validation_train_test_split(all_data, buckets_location=buckets_loc, bucket_number=i)
    #---------------------------------------------------
    # A value to compute the runtime of this testcase. 
    ts = time()
    # Create an instance of the Decision Tree object
    classifier = DecisionTree()
    # Train the Decision Tree using the training data.
    tree = classifier.build_tree(training_data, target)

    # Make the predicions for the whole testing dataset at once
    predictions = testing_data.apply(predict, axis='columns')
    # Compute the accuracy of the classification by checking the number of 
    # mismatches between the predictions and actual classification. 
    accuracy = accuracy_score(pd.DataFrame(testing_data, columns=[target]), pd.DataFrame(predictions, columns=[target]))
    # Increase the total accuracy to compute the average. 
    accuracy_nn += accuracy
    # A value to compute the runtime of this testcase
    te = time()
    # Print the accuracy of this testcase and the runtime. 
    print("No noise:       ", accuracy, "Took: ", te-ts, " seconds")
    #---------------------------------------------------
    
    # The rest of the for-loop works the same as the above, but it
    # uses the epsilon values as described in the paper. 
    
    #---------------------------------------------------
    ts = time()
    classifier = DecisionTree(1)
    tree = classifier.build_tree(training_data, target)

    predictions = testing_data.apply(predict, axis='columns')
    accuracy = accuracy_score(pd.DataFrame(testing_data, columns=[target]), pd.DataFrame(predictions, columns=[target]))
    accuracy_1 += accuracy
    te = time()
    print("Epsilon 1.000:  ", accuracy, "Took: ", te-ts, " seconds")
    #---------------------------------------------------
    ts = time()
    classifier = DecisionTree(0.75)
    tree = classifier.build_tree(training_data, target)

    predictions = testing_data.apply(predict, axis='columns')
    accuracy = accuracy_score(pd.DataFrame(testing_data, columns=[target]), pd.DataFrame(predictions, columns=[target]))
    accuracy_075 += accuracy
    te = time()
    print("Epsilon 0.750:  ", accuracy, "Took: ", te-ts, " seconds")
    #---------------------------------------------------
    classifier = DecisionTree(0.5)
    tree = classifier.build_tree(training_data, target)
    
    predictions = testing_data.apply(predict, axis='columns')
    accuracy = accuracy_score(pd.DataFrame(testing_data, columns=[target]), pd.DataFrame(predictions, columns=[target]))
    accuracy_050 += accuracy
    te = time()
    print("Epsilon 0.500:  ", accuracy, "Took: ", te-ts, " seconds")
    #---------------------------------------------------
    ts = time()
    classifier = DecisionTree(0.25)
    tree = classifier.build_tree(training_data, target)
    
    predictions = testing_data.apply(predict, axis='columns')
    accuracy = accuracy_score(pd.DataFrame(testing_data, columns=[target]), pd.DataFrame(predictions, columns=[target]))
    accuracy_025 += accuracy
    te = time()
    print("Epsilon 0.250:  ", accuracy, "Took: ", te-ts, " seconds")
    #---------------------------------------------------
    ts = time()
    classifier = DecisionTree(0.1)
    tree = classifier.build_tree(training_data, target)
    
    predictions = testing_data.apply(predict, axis='columns')
    accuracy = accuracy_score(pd.DataFrame(testing_data, columns=[target]), pd.DataFrame(predictions, columns=[target]))
    accuracy_010 += accuracy
    te = time()
    print("Epsilon 0.100:  ", accuracy, "Took: ", te-ts, " seconds")
    #---------------------------------------------------
    ts = time()
    classifier = DecisionTree(0.05)
    tree = classifier.build_tree(training_data, target)
    
    predictions = testing_data.apply(predict, axis='columns')
    accuracy = accuracy_score(pd.DataFrame(testing_data, columns=[target]), pd.DataFrame(predictions, columns=[target]))
    accuracy_005 += accuracy
    te = time()
    print("Epsilon 0.050:  ", accuracy, "Took: ", te-ts, " seconds")
    #---------------------------------------------------
    ts = time()
    classifier = DecisionTree(0.01)
    tree = classifier.build_tree(training_data, target)

    predictions = testing_data.apply(predict, axis='columns')
    accuracy = accuracy_score(pd.DataFrame(testing_data, columns=[target]), pd.DataFrame(predictions, columns=[target]))
    accuracy_001 += accuracy
    te = time()
    print("Epsilon 0.010:  ", accuracy, "Took: ", te-ts, " seconds")
    #---------------------------------------------------
    ts = time()
    classifier = DecisionTree(0.005)
    tree = classifier.build_tree(training_data, target)

    predictions = testing_data.apply(predict, axis='columns')
    accuracy = accuracy_score(pd.DataFrame(testing_data, columns=[target]), pd.DataFrame(predictions, columns=[target]))
    accuracy_0005 += accuracy
    te = time()
    print("Epsilon 0.005:  ", accuracy, "Took: ", te-ts, " seconds")
    #---------------------------------------------------
    ts = time()
    classifier = DecisionTree(0.001)
    tree = classifier.build_tree(training_data, target)

    predictions = testing_data.apply(predict, axis='columns')
    accuracy = accuracy_score(pd.DataFrame(testing_data, columns=[target]), pd.DataFrame(predictions, columns=[target]))
    accuracy_0001 += accuracy
    te = time()
    print("Epsilon 0.001:  ", accuracy, "Took: ", te-ts, " seconds")
    #---------------------------------------------------
    ts = time()
    classifier = DecisionTree(0.0000000001)
    tree = classifier.build_tree(training_data, target)

    predictions = testing_data.apply(predict, axis='columns')
    accuracy = accuracy_score(pd.DataFrame(testing_data, columns=[target]), pd.DataFrame(predictions, columns=[target]))
    accuracy_0 += accuracy
    te = time()
    print("Epsilon 0.000:  ", accuracy, "Took: ", te-ts, " seconds")

# Compute and print the average accuracy after 10-fold cross-validation.
print()
print('Average accuracy - No noise:       ', accuracy_nn/r)
print('Average accuracy - Epsilon 1.000:  ', accuracy_1/r)
print('Average accuracy - Epsilon 0.750:  ', accuracy_075/r)
print('Average accuracy - Epsilon 0.500:  ', accuracy_050/r)
print('Average accuracy - Epsilon 0.250:  ', accuracy_025/r)
print('Average accuracy - Epsilon 0.100:  ', accuracy_010/r)
print('Average accuracy - Epsilon 0.050:  ', accuracy_005/r)
print('Average accuracy - Epsilon 0.010:  ', accuracy_001/r)
print('Average accuracy - Epsilon 0.005:  ', accuracy_0005/r)
print('Average accuracy - Epsilon 0.001:  ', accuracy_0001/r)
print('Average accuracy - Epsilon 0.000:  ', accuracy_0/r)